In [1]:
from together import Together
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
class Agent:
    def __init__(self, system_prompt, model_name):
        self.system_prompt = system_prompt
        self.messages = []
        self.messages.append({
            "role": "system",
            "content": self.system_prompt
        })
        self.client_endpoint = Together()
        self.model_name = model_name

    def __call__(self, user_message=""):
        self.messages.append({
            "role": "user",
            "content": user_message
        })
        result = self.execute()
        self.messages.append({
            "role": "assistant",
            "content": result
        })
        return result

    def execute(self):
        stream = self.client_endpoint.chat.completions.create(
            model=self.model_name,
            messages=self.messages,
            stream=True,
            max_tokens=1024,
        )
        answer = ""
        for chunk in stream:
            answer += chunk.choices[0].delta.content
        return answer

In [3]:
# Test calling model
system_prompt = "You are an AI assistant."
model = "meta-llama/Llama-3.3-70B-Instruct-Turbo"
agent = Agent(system_prompt, model)
print(agent("Hi. Who are you?"))

Hello! I'm an artificial intelligence (AI) assistant, which means I'm a computer program designed to simulate conversation, answer questions, and provide information on a wide range of topics. I'm here to help you with any questions or tasks you may have, and I'll do my best to provide accurate and helpful responses. I don't have a personal name, but you can think of me as a helpful companion or a knowledgeable friend. How can I assist you today?


In [12]:
# Tools
def flight_planner(json_dict):
    # Simulate flight information tool
    start = json_dict.get("start_location", "")
    end = json_dict.get("end_location", "")
    flight_time = None
    flight_cost = None
    if start == "Singapore" and end == "Taipei":
        flight_time = 3
        flight_cost = 800
    elif start == "Taipei" and end == "Tokyo":
        flight_time = 6
        flight_cost = 200
    elif start == "Singapore" and end == "Tokyo":
        flight_time = 8
        flight_cost = 1500
    output = f"Flight time: {flight_time}, Flight Cost: {flight_cost}"
    return output

def calculate(json_dict):
    # Calculation tool
    operation = json_dict.get("operation", "")
    return eval(operation)

        

In [4]:
prompt = """
  You run in a loop of Thought, Action, PAUSE, Observation.
  At the end of the loop you output an Answer
  Use Thought to describe your thoughts about the question you have been asked.
  Use Action to run one of the actions available to you - then return PAUSE.
  Observation will be the result of running those actions.

  Your available actions are:

  flight_planner: Get flight time and cost between two cities.
  
  calculate: Runs a calculation on a mathematical expression and returns the result - uses Python so be sure to use floating point syntax if necessary
  USE calculate for EVERY CALCULATION
  
  The way you use the tools is by specifying a json blob.
  Specifically, this json should have an `action` key (with the name of the tool to use) and an `action_input` key (with the input to the tool going here).

  The only values that should be in the "action" field are:
    flight_planner: args: {"start_location": {"type": "string"}, "end_location": {"type": "string"}}
    example use :
    {{
      "action": "flight_planner",
      "action_input": {"start_location": "Beijing", "end_location", "Tokyo"}
    }}
    
    calculate:  args: {"operation": {"type": "string"}}

    ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:

$JSON_BLOB (inside markdown cell)

PAUSE
You will be called again with this:

Observation: the result of the action. This Observation is unique, complete, and the source of truth.

If you have the answer, output it as the Final Answer

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. 
Only use information provided by tools. If you don't know, say you don't know
"""


In [15]:
# Test model output
model = "meta-llama/Llama-3.3-70B-Instruct-Turbo"
agent = Agent(prompt, model)
# test_result = agent("What is the flight time and cost between Singapore and Tokyo?")
test_result = agent("Calculate 4 times 8")
print(test_result)

Thought: To calculate 4 times 8, I need to use the calculate action with the mathematical expression "4 * 8".

Action:
```json
{
  "action": "calculate",
  "action_input": {"operation": "4 * 8"}
}
```

PAUSE


In [18]:
# Setup regex function
import re
import json

def extract_json_block(text: str) -> dict:
    """
    Extracts the first JSON block enclosed in ```json ... ``` from the given text.
    
    Args:
        text (str): The input string containing a JSON block.
    
    Returns:
        dict: Parsed JSON object if found, otherwise None.
    """
    match = re.search(r'```json\s*(\{.*?\})\s*```', text, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return None
    else:
        print("No JSON block found.")
        return None
        
# Test regex function
print(extract_json_block(test_result))

{'action': 'calculate', 'action_input': {'operation': '4 * 8'}}


In [17]:
def agent_loop(max_iterations, system_prompt, model, query):
    agent = Agent(system_prompt, model)
    tools = ["flight_planner", "calculate"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(f"Iteration: {i}, result: {result}")
        if "PAUSE" in result:
            json_result = extract_json_block(result)
            chosen_tool = json_result.get("action", None)
            tool_input = json_result.get("action_input", None)

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}({tool_input})")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found!"
            print(next_prompt)
            continue
        if "Final Answer" in result:
            break

In [13]:
# Full test of agent
model = "meta-llama/Llama-3.3-70B-Instruct-Turbo"
query = "From Singapore, is it better to fly direct to Tokyo or have a stopover first at Taipei?"
agent_loop(max_iterations=10, system_prompt=prompt, model=model, query=query)

Iteration: 1, result: Thought: To determine whether it's better to fly direct to Tokyo or have a stopover at Taipei, we need to compare the flight times and costs of both options. We'll start by calculating the flight time and cost from Singapore to Tokyo with a stopover at Taipei, and then compare it to the direct flight from Singapore to Tokyo. First, let's get the flight time and cost from Singapore to Taipei.

Action:
```json
{
  "action": "flight_planner",
  "action_input": {"start_location": "Singapore", "end_location": "Taipei"}
}
```
PAUSE
Observation: Flight time: 3, Flight Cost: 800
Iteration: 2, result: Thought: Now that we have the flight time and cost from Singapore to Taipei, we need to get the flight time and cost from Taipei to Tokyo to calculate the total flight time and cost for the stopover option.

Action:
```json
{
  "action": "flight_planner",
  "action_input": {"start_location": "Taipei", "end_location": "Tokyo"}
}
```
PAUSE
Observation: Flight time: 6, Flight Co